# Scraping an HTML table into a Pandas dataframe
With basically two statements!

In [1]:
import pandas as pd
import requests # a user-friendly web package

Step 1: Download a web page.  (The example below is good because the raw data is potentially very useful for further computation, but the page is larded with graphics and ads and whatnot.  Simple copy/paste not likely to work here.)

In [2]:
page = requests.get("https://www.nationmaster.com/country-info/stats/Media/Internet-users")

# N.B.: page content is also cached locally:
# page_content = open("scraping-example-cached.html", mode="rb").read()

The response is an object that can be examined.  200 = OK.

In [3]:
type(page)

requests.models.Response

In [4]:
page.status_code

200

In [5]:
page.content[:1000]

b'\n<!DOCTYPE HTML>\n<html>\n    <head>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    \n    \n<script type="text/javascript">;window._taboola=window._taboola||[];_taboola.push({category:\'auto\'});!function(e,f,u,i){if(!document.getElementById(i)){e.async=1;e.src=u;e.id=i;f.parentNode.insertBefore(e,f);}}(document.createElement(\'script\'),document.getElementsByTagName(\'script\')[0],\'//cdn.taboola.com/libtrc/nationmaster/loader.js\',\'tb_loader_script\');if(window.performance&&typeof window.performance.mark==\'function\'){window.performance.mark(\'tbl_ic\');}</script>\n\n        <title>Countries Compared by Media &gt; Internet users. International Statistics at NationMaster.com</title>\n        <meta name="description" content="" />\n        <meta name="author" content="">\n        <meta name="viewport" content="width=device-width,initial-scale=1, user-scalable=no">\n        \n        <meta property="og:type" content="web

Step 2: Have Pandas scan for tables and return a list of auto-constructed dataframes, one per table.  Lots of options here, check the documentation.  Caution: tables in the web page don't necessarily correspond to what you visually recognize as tables.  You need to look at the results, locate the dataframe you're after, then clean it up.

In [6]:
frames = pd.read_html(page.content)
len(frames)

2

In [7]:
frames[0].head()

,#,COUNTRY,AMOUNT,DATE,GRAPH,HISTORY
0,1,China,389 million,2009,NaN,NaN
1,2,United States,245 million,2009,NaN,NaN
2,3,Japan,99.18 million,2009,NaN,NaN
3,NaN,Group of 7 countries (G7) average (profile),80.32 million,2009,NaN,NaN
4,4,Brazil,75.98 million,2009,NaN,NaN


In [8]:
frames[1]

,STAT,COUNTRIES COVERED
0,2008,17
1,Per 100 people,189


In [9]:
# this is the dataframe we want
df = frames[0]

Start cleaning the data.

In [10]:
df.set_index("COUNTRY", inplace=True)
df

,#,AMOUNT,DATE,GRAPH,HISTORY
COUNTRY,,,,,
China,1,389 million,2009,NaN,NaN
United States,2,245 million,2009,NaN,NaN
Japan,3,99.18 million,2009,NaN,NaN
Group of 7 countries (G7) average (profile),NaN,80.32 million,2009,NaN,NaN
Brazil,4,75.98 million,2009,NaN,NaN
...,...,...,...,...,...
Niue,214,1100,2009,NaN,NaN
"Saint Helena, Ascension, and Tristan da Cunha",=215,900,2009,NaN,NaN
Saint Helena,=215,900,2009,NaN,NaN


Let's discard the rows that represent aggregates.

In [11]:
df = df.loc[~df["#"].isna()].copy() # copy not necessary, but silences spurious warning
df

,#,AMOUNT,DATE,GRAPH,HISTORY
COUNTRY,,,,,
China,1,389 million,2009,NaN,NaN
United States,2,245 million,2009,NaN,NaN
Japan,3,99.18 million,2009,NaN,NaN
Brazil,4,75.98 million,2009,NaN,NaN
Germany,5,65.12 million,2010,NaN,NaN
...,...,...,...,...,...
Niue,214,1100,2009,NaN,NaN
"Saint Helena, Ascension, and Tristan da Cunha",=215,900,2009,NaN,NaN
Saint Helena,=215,900,2009,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218 entries, China to Christmas Island
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   #        218 non-null    object 
 1   AMOUNT   218 non-null    object 
 2   DATE     218 non-null    int64  
 3   GRAPH    0 non-null      float64
 4   HISTORY  0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 10.2+ KB


The `AMOUNT` column is text ("object") because some column values contain the word "million" (note that Pandas made the entire column text, even those values that don't contain "million").  Let's convert this to numeric.

First, identify and select just those values needing conversion.  Then we'll incrementally build up a transformation.

In [13]:
subset = df.loc[df.AMOUNT.str.contains("million"), "AMOUNT"]
subset

COUNTRY
China                      389 million
United States              245 million
Japan                    99.18 million
Brazil                   75.98 million
Germany                  65.12 million
                             ...      
Moldova                   1.29 million
Paraguay                   1.1 million
Bolivia                    1.1 million
Kuwait                     1.1 million
Republic of Macedonia     1.06 million
Name: AMOUNT, Length: 97, dtype: object

So we see that 97 of the 218 rows need converting.  Extract just the numeric portion of each value.

In [14]:
subset.str.split(" ")

COUNTRY
China                      [389, million]
United States              [245, million]
Japan                    [99.18, million]
Brazil                   [75.98, million]
Germany                  [65.12, million]
                               ...       
Moldova                   [1.29, million]
Paraguay                   [1.1, million]
Bolivia                    [1.1, million]
Kuwait                     [1.1, million]
Republic of Macedonia     [1.06, million]
Name: AMOUNT, Length: 97, dtype: object

In [15]:
subset.str.split(" ").str.get(0)

COUNTRY
China                      389
United States              245
Japan                    99.18
Brazil                   75.98
Germany                  65.12
                         ...  
Moldova                   1.29
Paraguay                   1.1
Bolivia                    1.1
Kuwait                     1.1
Republic of Macedonia     1.06
Name: AMOUNT, Length: 97, dtype: object

Here's the transformation we want.

In [16]:
revised_subset = subset.str.split(" ").str.get(0).astype(float)*1e6
revised_subset

COUNTRY
China                    389000000.0
United States            245000000.0
Japan                     99180000.0
Brazil                    75980000.0
Germany                   65120000.0
                            ...     
Moldova                    1290000.0
Paraguay                   1100000.0
Bolivia                    1100000.0
Kuwait                     1100000.0
Republic of Macedonia      1060000.0
Name: AMOUNT, Length: 97, dtype: float64

The following won't update the dataframe.  It will update `subset`, not `df`.

In [17]:
# won't work!!!
# subset = revised_subset

But this does:

In [18]:
df.loc[df.AMOUNT.str.contains("million"), "AMOUNT"] = revised_subset
df

,#,AMOUNT,DATE,GRAPH,HISTORY
COUNTRY,,,,,
China,1,389000000.0,2009,NaN,NaN
United States,2,245000000.0,2009,NaN,NaN
Japan,3,99180000.0,2009,NaN,NaN
Brazil,4,75980000.0,2009,NaN,NaN
Germany,5,65120000.0,2010,NaN,NaN
...,...,...,...,...,...
Niue,214,1100,2009,NaN,NaN
"Saint Helena, Ascension, and Tristan da Cunha",=215,900,2009,NaN,NaN
Saint Helena,=215,900,2009,NaN,NaN


Almost done.  But the values in the `AMOUNT` column we *didn't* replace still have type text (due to the way Pandas originally constructed the column).  Simplest to just convert the entire column to float.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218 entries, China to Christmas Island
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   #        218 non-null    object 
 1   AMOUNT   218 non-null    object 
 2   DATE     218 non-null    int64  
 3   GRAPH    0 non-null      float64
 4   HISTORY  0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 10.2+ KB


In [20]:
df.AMOUNT = df.AMOUNT.astype(float)
df

,#,AMOUNT,DATE,GRAPH,HISTORY
COUNTRY,,,,,
China,1,389000000.0,2009,NaN,NaN
United States,2,245000000.0,2009,NaN,NaN
Japan,3,99180000.0,2009,NaN,NaN
Brazil,4,75980000.0,2009,NaN,NaN
Germany,5,65120000.0,2010,NaN,NaN
...,...,...,...,...,...
Niue,214,1100.0,2009,NaN,NaN
"Saint Helena, Ascension, and Tristan da Cunha",=215,900.0,2009,NaN,NaN
Saint Helena,=215,900.0,2009,NaN,NaN


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218 entries, China to Christmas Island
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   #        218 non-null    object 
 1   AMOUNT   218 non-null    float64
 2   DATE     218 non-null    int64  
 3   GRAPH    0 non-null      float64
 4   HISTORY  0 non-null      float64
dtypes: float64(3), int64(1), object(1)
memory usage: 10.2+ KB


Here's the payoff:

In [22]:
df.AMOUNT.describe()

count    2.180000e+02
mean     8.523142e+06
std      3.335812e+07
min      4.640000e+02
25%      8.730000e+04
50%      7.082000e+05
75%      4.427500e+06
max      3.890000e+08
Name: AMOUNT, dtype: float64

## Using BeautifulSoup for more control
You can use BeautifulSoup, an HTML parser, for greater control in selecting which table to pass to Pandas.

In [23]:
from bs4 import BeautifulSoup

In [24]:
soup = BeautifulSoup(page.content)

Returns an object.

In [25]:
type(soup)

bs4.BeautifulSoup

HTML elements can be found various ways.

In [26]:
soup.title

<title>Countries Compared by Media &gt; Internet users. International Statistics at NationMaster.com</title>

In [27]:
tables = soup.find_all("table")

In [28]:
len(tables)

2

Each HTML element is actually an object that must be converted to a string before passing to Pandas.

In [29]:
type(tables[0])

bs4.element.Tag

Note that Pandas will still return a list even if there's only one dataframe.

In [30]:
frames = pd.read_html(str(tables[0]))

Now proceed as before.